In [1]:
from pymatgen.core import Structure
import json
from ase.data import chemical_symbols
import numpy as np

In [3]:
# with open('/Users/treents/project/aiida-cwf/data/wien2k/for central volumes/oxides_pbe_prec3.json', 'r') as f:
#     oxide_volumes = json.load(f)
#     oxide_volumes = {
#         k: v['min_volume'] for k, v in oxide_volumes['BM_fit_data'].items()
#     }

with open('./central_volumes_oxides-verification-PBE-missing-actinides-v1.json', 'r') as f:
    oxide_volumes_PBE = json.load(f)
    
with open('./central_volumes_oxides-verification-PBEsol-v1.json', 'r') as f:
    oxide_volumes_PBEsol = json.load(f)

with open('./central_volumes_oxides-verification-LDA-v1.json', 'r') as f:
    oxide_volumes_LDA = json.load(f)
    
oxide_volumes = {
    'LDA': oxide_volumes_LDA,
    'PBE': oxide_volumes_PBE,
    'PBEsol': oxide_volumes_PBEsol
}

In [4]:
prototypes = {
    'X2O': 'xsfs-oxides-verification-PBE-v1/Ac-X2O.xsf',
    'X2O3': 'xsfs-oxides-verification-PBE-v1/Ac-X2O3.xsf',
    'XO': 'xsfs-oxides-verification-PBE-v1/Ac-XO.xsf',
    'X2O5': 'xsfs-oxides-verification-PBE-v1/Ac-X2O5.xsf',
    'XO2': 'xsfs-oxides-verification-PBE-v1/Ac-XO2.xsf',
    'XO3': 'xsfs-oxides-verification-PBE-v1/Ac-XO3.xsf',
}

In [5]:
assert set(prototypes.keys()) == set(['X2O', 'X2O5', 'XO2', 'X2O3', 'XO', 'XO3'])

In [6]:
for funct, vols in oxide_volumes.items():
    suffix = '-missing-actinides' if funct == 'PBE' else ''
    for Z in range(1, 103+1):
        element_name = chemical_symbols[Z]
        for conf in ['X2O', 'X2O5', 'XO2', 'X2O3', 'XO', 'XO3']:
            if element_name not in vols[conf]:
                continue
            new_strct = Structure.from_file(prototypes[conf]).replace_species(
                {'Ac': element_name}
                ).get_sorted_structure().scale_lattice(
                    vols[conf][element_name]
                ).to_file(
                    f'xsfs-oxides-verification-{funct}{suffix}-v1/{element_name}-{conf}.xsf'
            )
        

/Users/treents/.aiida_venvs/aiida-cwf/lib/python3.11/site-packages/pymatgen/core/composition.py:326: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  return sum((el.X * abs(amt) for el, amt in self.items())) / self.num_atoms
/Users/treents/.aiida_venvs/aiida-cwf/lib/python3.11/site-packages/pymatgen/core/composition.py:326: UserWarning: No Pauling electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  return sum((el.X * abs(amt) for el, amt in self.items())) / self.num_atoms
/Users/treents/.aiida_venvs/aiida-cwf/lib/python3.11/site-packages/pymatgen/core/composition.py:326: UserWarning: No Pauling electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  return sum((el.X * abs(amt) for el

In [7]:
for Z in range(1, 96+1):
    element_name = chemical_symbols[Z]
    for conf in ['X2O', 'X2O5', 'XO2', 'X2O3', 'XO', 'XO3']:
        strct = Structure.from_file(f'test_structures_TR/{element_name}_{conf}.xsf')
        ref_strct = Structure.from_file(f'xsfs-oxides-verification-PBE-v1/{element_name}-{conf}.xsf')
        
        assert np.isclose(strct.volume, ref_strct.volume, atol=1e-7), (
            f'{strct.volume} != {ref_strct.volume}'
        )
        assert np.allclose(strct.lattice.abc, ref_strct.lattice.abc, atol=1e-7)
        assert np.allclose(strct.lattice.angles, ref_strct.lattice.angles, atol=1e-7)
        assert strct.formula == ref_strct.formula, (
            f'{strct.formula} != {ref_strct.formula}'
        )
        assert np.allclose(strct.frac_coords, ref_strct.frac_coords, atol=1e-7), (
            f'{strct.frac_coords} != {ref_strct.frac_coords}'
        )
        assert np.allclose(strct.cart_coords, ref_strct.cart_coords, atol=1e-7)

/Users/treents/.aiida_venvs/aiida-cwf/lib/python3.11/site-packages/pymatgen/core/composition.py:370: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms = sorted(sym_amt, key=lambda sym: get_el_sp(sym).X)
/Users/treents/.aiida_venvs/aiida-cwf/lib/python3.11/site-packages/pymatgen/core/composition.py:370: UserWarning: No Pauling electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms = sorted(sym_amt, key=lambda sym: get_el_sp(sym).X)
/Users/treents/.aiida_venvs/aiida-cwf/lib/python3.11/site-packages/pymatgen/core/composition.py:370: UserWarning: No Pauling electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms = sorted(sym_amt, key=lambda sym: get_el_sp(sym).X)


In [ ]:
strct.get_sorted_structure()

Structure Summary
Lattice
    abc : 3.524642686284117 3.524642686284112 3.52464268628412
 angles : 59.99999999999993 59.999999999999986 59.99999999999993
 volume : 30.962091653893133
      A : 3.05243010558507 0.0 1.76232134314206
      B : 1.01747670186169 2.87785870234289 1.76232134314206
      C : 0.0 0.0 3.52464268628412
    pbc : True True True
PeriodicSite: F (1.017, 0.7195, 1.762) [0.25, 0.25, 0.25]
PeriodicSite: F (3.052, 2.158, 5.287) [0.75, 0.75, 0.75]
PeriodicSite: O (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]

In [9]:
element_name, conf, Z, strct

('F',
 'X2O',
 9,
 Structure Summary
 Lattice
     abc : 3.524642686284117 3.524642686284112 3.52464268628412
  angles : 59.99999999999993 59.999999999999986 59.99999999999993
  volume : 30.962091653893133
       A : 3.05243010558507 0.0 1.76232134314206
       B : 1.01747670186169 2.87785870234289 1.76232134314206
       C : 0.0 0.0 3.52464268628412
     pbc : True True True
 PeriodicSite: F (1.017, 0.7195, 1.762) [0.25, 0.25, 0.25]
 PeriodicSite: F (3.052, 2.158, 5.287) [0.75, 0.75, 0.75]
 PeriodicSite: O (0.0, 0.0, 0.0) [0.0, 0.0, 0.0])

In [ ]:
strct.coo

(2.5672695406806296, 2.5672695406806327, 2.56726954068063)